In [1]:
from ultralytics import YOLO
import glob
import os
from IPython.display import display, Image
import shutil
import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from pandas.errors import EmptyDataError
import random

import warnings
from sklearn.exceptions import DataConversionWarning, InconsistentVersionWarning
warnings.filterwarnings(action = 'ignore', category = DataConversionWarning)
warnings.filterwarnings(action = 'ignore', category = FutureWarning)
warnings.filterwarnings(action = 'ignore', category = InconsistentVersionWarning)
warnings.filterwarnings(action = "ignore", category = DeprecationWarning)
pd.options.mode.chained_assignment = None  # default='warn'

import gc
import torch

with torch.no_grad():
    torch.cuda.empty_cache()
torch.cuda.empty_cache()
gc.collect()

0

In [2]:
%%writefile 'data.yaml'
        
names:
    0: свободно стоящая типа «рюмка»
    1: одноцепная башенного типа
    2: портальная на оттяжках
    3: двухцепная башенного типа
    4: одноцепная на оттяжках
    5: на оттяжках типа «набла»
    6: многогранные опоры ЛЭП
    7: необычные опоры ЛЭП

train: ./data/images/train/
val: ./data/images/val/

Overwriting data.yaml


In [3]:
js = '/home/vlad/.config/Ultralytics/settings.json'
os.remove(js)

In [4]:
import yaml

with open("data.yaml", 'r') as f:
    dataset_yaml = yaml.safe_load(f)

dataset_yaml["train"] = "./data/images/train/"
dataset_yaml["val"]   = "./data/images/val/"
#dataset_yaml["test"]  = "../data/test/images"

with open("data.yaml", 'w') as f:
    yaml.dump(dataset_yaml, f)
    
yaml

<module 'yaml' from '/home/vlad/anaconda3/lib/python3.11/site-packages/yaml/__init__.py'>

In [5]:
def seed_everything(seed):
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) 

In [6]:
import gc
import ctypes

def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

clean_memory()

In [7]:
from ultralytics import settings

# Update a setting
settings.update({"mlflow": True})

# Reset settings to default values
#settings.reset()

In [8]:
optimizer = 'AdamW',
lr0 = 0.001,

In [9]:
model = YOLO('yolo11s.pt')

imgsz = 1280
name = f'train_95_{imgsz}s2'

directory = f'./runs/detect/{name}s'
shutil.rmtree(directory, ignore_errors = True)

results = model.train(
    warmup_epochs = 0,
    resume = False,
    
    task = 'detect',
    data = 'data.yaml',
    
    batch = 5,
    workers = 6,
    
    epochs = 10000,
    imgsz = imgsz,
    plots = True,
    
    optimizer = 'AdamW',
    lr0 = 0.001,
    
    patience = 20,
    name = name,
    save_dir = f'./runs/detect/{name}',
    single_cls = False,
    augment = True,
    dropout = 0.03,
    int8 = False,
    
    cos_lr = True,
    fraction = 1,
    weight_decay = 0.0005, # 0.0005
    box = 7.5, 
    cls = 0.5,
    dfl = 1.5,
    
    ### RandomPerspective
    degrees = 0.25,     # Максимальный абсолютный диапазон градусов для произвольных поворотов.
    translate = 0.1,    # Максимальный перевод в виде доли от размера изображения.
    scale = 0.5,        # Диапазон коэффициентов масштабирования, например, scale=0.1 означает 0.9-1.1.
    shear = 0.25,       # Максимальный угол сдвига в градусах.
    #perspective = 0.0003, # Коэффициент искажения перспективы.
    ### RandomPerspective
    
    ### albumentations
    #hsv_h = 0.015,     # Регулирует оттенок изображения на долю цветового круга, внося вариативность цвета.
    #hsv_s = 0.7,       # Изменяет насыщенность изображения, влияя на интенсивность цветов.  
    #hsv_v = 0.4,       # Изменяет значение (яркость) изображения на небольшую долю
    flipud = 0.5,       # Переворачивает изображение вверх ногами с заданной вероятностью
    fliplr = 0.5,       # Переворачивает изображение слева направо с заданной вероятностью
    bgr = 0.5,          # Переключает каналы изображения с RGB на BGR с заданной вероятностью
    mosaic = 0.25,      # Объединяет четыре обучающих изображения в одно
    mixup = 0.25,       # Смешивает два изображения и их надписи, создавая составное изображение
    copy_paste = 0.25,  # Копирует объекты с одного изображения и вставляет их на другое
    erasing = 0.25,     # Случайное стирание части изображения во время обучения   
    ### albumentations

)

Ultralytics 8.3.39 🚀 Python-3.11.5 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=10000, time=None, patience=20, batch=5, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=6, project=None, name=train_95_1280s2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.03, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /home/vlad/Games/00-LEP/02/datasets/data/labels/train... 578 ima

train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_0_156E05AA993_DSC00243.JPG: ignoring corrupt image/label: labels require 5 columns, 1 columns detected
train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_0_156E05AA993_DSC00358.JPG: ignoring corrupt image/label: image file is truncated (70 bytes not processed)
train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_2_156E05AA993_DSC00208.JPG: ignoring corrupt image/label: labels require 5 columns, 1 columns detected
train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_2_156E05AA993_DSC00384.JPG: ignoring corrupt image/label: labels require 5 columns, 1 columns detected
train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_2_156E05AA993_DSC00386.JPG: ignoring corrupt image/label: labels require 5 columns, 1 columns detected
train: WARNING ⚠️ /home/vlad/Games/00-LEP/02/datasets/data/images/train/alb_2_156E05AA993_DSC01425.JP

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/vlad/Games/00-LEP/02/datasets/data/labels/val.cache... 273 i


...

  124/10000      5.77G     0.7242      0.625     0.9799          7       1280: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        300        442      0.864      0.732      0.812       0.62
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 104, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



124 epochs completed in 1.986 hours.
Optimizer stripped from runs/detect/train_95_1280s2/weights/last.pt, 19.3MB
Optimizer stripped from runs/detect/train_95_1280s2/weights/best.pt, 19.3MB

Validating runs/detect/train_95_1280s2/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.11.5 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
YOLO11s summary (fused): 238 layers, 9,415,896 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        300        442      0.891      0.788      0.877      0.671
свободно стоящая типа «рюмка»        222        351      0.961        0.9      0.962      0.818
одноцепная башенного типа         15         26      0.757      0.885      0.909      0.792
портальная на оттяжках         18         18          1      0.694      0.953      0.835
двухцепная башенного типа         34         41      0.826      0.585      0.776       0.61
одноцепная на оттяжках          3          3      0.815          1      0.995      0.573
   необычные опоры ЛЭП          2          3      0.987      0.667      0.666      0.401
Speed: 0.5ms preprocess, 37.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train_95_1280s2
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
